In [12]:
import pyspark
from delta import *
from pyspark.sql.types import *
from delta.tables import *
from pyspark.sql.functions import *

#  Create a spark session with Delta
builder = pyspark.sql.SparkSession.builder.appName("SparkDelta") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

# Create spark context
spark_delta = configure_spark_with_delta_pip(builder).getOrCreate()
spark_delta.sparkContext.setLogLevel("ERROR")

In [13]:
# Leitura dos dados

try:
    df_top_tracks = spark_delta.read.option("multiline", "false").json('c:/Users/benedito.aires/Documents/GitHub/lastfm-pipeline/data/in/getTopTracks.json')
    df_top_artist = spark_delta.read.option("multiline", "false").json('c:/Users/benedito.aires/Documents/GitHub/lastfm-pipeline/data/in/gettopartists.json')

except Exception as ex:
    print(ex)

In [14]:
df_selecao = df_top_artist.select(
    col('corpo.topartists.artist.name').cast('string').alias('name'),
    col('corpo.topartists.artist.playcount').alias('playcount'),
    col('corpo.topartists.artist.@attr.rank').alias('rank')
)

df_selecao_top_tracks = df_top_tracks.select(
    col('corpo.recenttracks.track.artist.#text').cast('string').alias('artista'),
    col('corpo.recenttracks.track.album.#text').alias('album'),
    col('corpo.recenttracks.track.name').alias('musica'),
    col('corpo.recenttracks.track.date.#text').alias('data_execucao')

    
).dropDuplicates()


In [15]:
df_selecao_top_tracks.show()

+--------------------+--------------------+--------------------+--------------------+
|             artista|               album|              musica|       data_execucao|
+--------------------+--------------------+--------------------+--------------------+
|[Edguy, DragonFor...|[Mandrake, Inhuma...|[All the Clowns, ...|[13 May 2023, 19:...|
|[Fernando Mendes,...|[Selecao de Ouro,...|[Roda Gigante, Ro...|[28 Oct 2023, 16:...|
|[Jemina Pearl, Je...|[Break It Up, Bre...|[No Good, Retrogr...|[12 Feb 2023, 03:...|
|[The Cure, The Be...|[The Head on the ...|[Inbetween Days, ...|[29 Jan 2023, 18:...|
|[Labrinth, Neon T...|[Ends & Begins, P...|[The Feels, Every...|[22 Sep 2023, 15:...|
|[The Smiths, The ...|[Strangeways, Her...|[Unhappy Birthday...|[16 Aug 2023, 11:...|
|[Marcelo Jeneci, ...|[Guaia, Umas e Ou...|[Aí Sim, Yolanda,...|[22 Apr 2024, 22:...|
|[Whitesnake, Sona...|[Whitesnake (2018...|[Is This Love - 2...|[25 Jun 2023, 17:...|
|[Chicago, Electri...|[Chicago 16 (Expa...|[Hard to Sa

In [16]:
df_selecao.show()

+--------------------+--------------------+--------------------+
|                name|           playcount|                rank|
+--------------------+--------------------+--------------------+
|[Stratovarius, Vo...|[2183, 999, 830, ...|[1, 2, 3, 4, 5, 6...|
|[No Doubt, Beast ...|[60, 58, 57, 57, ...|[51, 52, 53, 54, ...|
|[REO Speedwagon, ...|[38, 37, 37, 36, ...|[101, 102, 103, 1...|
|[Kid Abelha, Raça...|[21, 21, 21, 21, ...|[151, 152, 153, 1...|
|[Nilton Cesar, Ti...|[15, 15, 15, 14, ...|[201, 202, 203, 2...|
|[R.E.M., Renato L...|[11, 11, 11, 11, ...|[251, 252, 253, 2...|
|[Eminem, Forrozão...|[7, 7, 7, 7, 7, 7...|[301, 302, 303, 3...|
|[Asleep at the Wh...|[5, 5, 5, 5, 5, 5...|[351, 352, 353, 3...|
|[Damien Rice, Dea...|[4, 4, 4, 4, 4, 4...|[401, 402, 403, 4...|
|[Avalanch, Baco E...|[3, 3, 3, 3, 3, 3...|[451, 452, 453, 4...|
|[Matanza, Matogro...|[3, 3, 3, 3, 3, 3...|[501, 502, 503, 5...|
|[Willie Nelson, Y...|[3, 3, 3, 3, 3, 3...|[551, 552, 553, 5...|
|[Dimas e Seus Tec...|[2,

In [17]:
df_top_artistas = df_selecao.select(
    posexplode(split("name", ",")).alias("pos_artista", "artista"),
    'playcount',
    'rank'    
).select(
    'pos_artista',
    'artista',
    expr("playcount[pos_artista]").cast('int').alias('playcount'),
    expr("rank[pos_artista]").cast('int').alias('rank')

).drop('pos_artista')

In [18]:
df_top_tracks_s = df_selecao_top_tracks.select(
    posexplode(split("artista", ",")).alias("pos_artista", "artista"),
    'album',
    'musica',
    'data_execucao'    
).select(
    'pos_artista',
    'artista',
    expr("album[pos_artista]").alias('album'),
    expr("musica[pos_artista]").alias('musica'),
    expr("data_execucao[pos_artista]").alias('data_execucao') 
).drop('pos_artista')

In [19]:
df_top_artistas.show()

+----------------+---------+----+
|         artista|playcount|rank|
+----------------+---------+----+
|   [Stratovarius|     2183|   1|
|         Volbeat|      999|   2|
|           Keane|      830|   3|
|       Dreamtale|      781|   4|
|           Edguy|      507|   5|
|  Blind Guardian|      388|   6|
|       Green Day|      381|   7|
|       Helloween|      370|   8|
| Shakin' Stevens|      331|   9|
|        The Cure|      324|  10|
|      The Smiths|      317|  11|
|         Sabaton|      231|  12|
|  Sonata Arctica|      213|  13|
|       Nightwish|      196|  14|
|      Whitesnake|      196|  15|
|          Travis|      195|  16|
|     DragonForce|      180|  17|
|     The Beatles|      167|  18|
|    Los Hermanos|      161|  19|
|      Maria Gadú|      161|  20|
+----------------+---------+----+
only showing top 20 rows



In [20]:
df_top_tracks_s.show()

+--------------------+--------------------+--------------------+------------------+
|             artista|               album|              musica|     data_execucao|
+--------------------+--------------------+--------------------+------------------+
|              [Edguy|            Mandrake|      All the Clowns|13 May 2023, 19:45|
|         DragonForce|     Inhuman Rampage|Through the Fire ...|13 May 2023, 19:35|
|             Sabaton|Coat Of Arms (Bon...|  The Final Solution|13 May 2023, 19:32|
|             Sabaton|              Heroes|    To Hell and Back|13 May 2023, 19:28|
|             Sabaton|           Metalizer|The Hammer Has Fa...|13 May 2023, 19:20|
|             Ednardo|O Romance Do Pavã...|    Pavão Mysteriozo|13 May 2023, 17:49|
|     O Teatro Mágico|Entrada para Raro...|   O anjo mais velho|13 May 2023, 17:44|
|           Fábio Jr.|Fábio Jr. 2002 (A...|    20 e Poucos Anos|13 May 2023, 17:41|
|    Flavio Venturini|Bis - Flavio Vent...|           Espanhola|13 May 2023,

In [24]:
try:
    df_top_artistas.write.format("delta").mode("overwrite").save('c:/Users/benedito.aires/Documents/GitHub/lastfm-pipeline/data/out/top_artists/')

except Exception as ex:
    print(ex)

An error occurred while calling o224.save.
: com.google.common.util.concurrent.ExecutionError: java.lang.UnsatisfiedLinkError: org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Ljava/lang/String;I)Z
	at com.google.common.cache.LocalCache$Segment.get(LocalCache.java:2261)
	at com.google.common.cache.LocalCache.get(LocalCache.java:4000)
	at com.google.common.cache.LocalCache$LocalManualCache.get(LocalCache.java:4789)
	at org.apache.spark.sql.delta.DeltaLog$.getDeltaLogFromCache$1(DeltaLog.scala:790)
	at org.apache.spark.sql.delta.DeltaLog$.apply(DeltaLog.scala:800)
	at org.apache.spark.sql.delta.DeltaLog$.forTable(DeltaLog.scala:677)
	at org.apache.spark.sql.delta.sources.DeltaDataSource.createRelation(DeltaDataSource.scala:189)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:48)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:75)
	at org.apache.spark.sql.execution.

In [22]:
# df_top_tracks_s.write.format('delta').mode('overwrite','true').save('c:/Users/benedito.aires/Documents/GitHub/lastfm-pipeline/data/out/')